In [1]:
import corrcal
import numpy as np
import matplotlib.pyplot as plt
import ctypes
import numba

The first goal in this notebook is to investigate the `corrcal.cfuncs.apply_gains_to_matrix` function (`apply_gains_to_mat` function in the C code). This involves the following:

* Simulate some data and build the covariance matrix.
* Simulate per-antenna gains.
* Apply the gains using `numpy`, basic python (with and without `numba`), and `corrcal`.
* Compare the results.

In [2]:
# First, make the antenna arrays.
ant1 = np.array([0, 1])
ant2 = np.array([1, 2])
ants = set(ant1).union(set(ant2))

# Make the gains here because it's easy.
gains = np.array(
    list(
        np.random.uniform() + 1j * np.random.uniform()
        for ant in ants
    )
)

# Simulate the data
data = np.array(
    list(
        np.random.uniform() + 1j * np.random.uniform()
        for ant in ant1
    )
)

# Make the complex-valued covariance
cov = np.outer(data, np.conj(data))

In [3]:
# Step 1: brute force
def apply_gains_simple(mat, gains, ant1, ant2):
    out = np.zeros_like(mat)
    for i, ai in enumerate(ant1):
        for j, aj in enumerate(ant2):
            out[i,j] = mat[i,j] * gains[ai] * np.conj(gains[aj])
    return out

# Step 2: numba-fied brute force
@numba.njit
def apply_gains_numba(mat, gains, ant1, ant2):
    out = np.zeros_like(mat)
    for i, ai in enumerate(ant1):
        for j, aj in enumerate(ant2):
            out[i,j] = mat[i,j] * gains[ai] * np.conj(gains[aj])
    return out

# Step 3: corrcal version
def apply_gains_corrcal(mat, gains, ant1, ant2):
    out = mat.copy()
    corrcal.cfuncs.apply_gains_to_matrix(
        out.ctypes.data,
        gains.ctypes.data,
        ant1.ctypes.data,
        ant2.ctypes.data,
        len(ant1),
        len(ant1),
    )
    return out

In [4]:
# Check that all results are equivalent
simple_numba_match = np.allclose(
    apply_gains_simple(cov, gains, ant1, ant2),
    apply_gains_numba(cov, gains, ant1, ant2),
)
simple_corrcal_match = np.allclose(
    apply_gains_simple(cov, gains, ant1, ant2),
    apply_gains_corrcal(cov, gains, ant1, ant2),
)
simple_numba_match, simple_corrcal_match

(True, False)

In [5]:
apply_gains_simple(cov, gains, ant1, ant2)

array([[0.88155161-0.16803676j, 0.36537187-0.63405702j],
       [0.44901293+0.5112678j , 0.55483831-0.00529033j]])

In [6]:
apply_gains_corrcal(cov, gains, ant1, ant2)

array([[0.88155161-0.16803676j, 0.41128338-0.47741187j],
       [0.62340162+0.48560777j, 0.55483831-0.00529033j]])

In [7]:
cov

array([[0.97784776+0.j        , 0.56817934-0.64695643j],
       [0.56817934+0.64695643j, 0.75817568+0.j        ]])

In [8]:
# Now do timing tests

In [9]:
%%timeit
_ = apply_gains_simple(cov, gains, ant1, ant2)

14.3 µs ± 227 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
%%timeit
_ = apply_gains_numba(cov, gains, ant1, ant2)

660 ns ± 12 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
%%timeit
_ = apply_gains_corrcal(cov, gains, ant1, ant2)

6.54 µs ± 506 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
# Do the dense versions. First, setup data.
cov_dense = np.zeros((2 * len(data), 2 * len(data)), dtype=float)
cov_dense[::2,::2] = cov.real
cov_dense[1::2,1::2] = cov.imag

gains_dense = np.zeros(2 * len(gains), dtype=float)
gains_dense[::2] = gains.real
gains_dense[1::2] = gains.imag

In [13]:
cov

array([[0.97784776+0.j        , 0.56817934-0.64695643j],
       [0.56817934+0.64695643j, 0.75817568+0.j        ]])

In [14]:
cov_dense

array([[ 0.97784776,  0.        ,  0.56817934,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.64695643],
       [ 0.56817934,  0.        ,  0.75817568,  0.        ],
       [ 0.        ,  0.64695643,  0.        ,  0.        ]])

If $z = a + ib$ and $w = c + id$, then $zw = ac - bd + i(bc + ad)$ and $z\bar{w} = ac + bd + i(bc - ad)$.

In [15]:
def apply_gains_simple_dense(mat, gains, ant1, ant2):
    out = np.zeros_like(mat)
    for i, ai in enumerate(ant1):
        for j, aj in enumerate(ant2):
            re_i = 2 * ai
            im_i = 2 * ai + 1
            re_j = 2 * aj
            im_j = 2 * aj + 1
            re_ij = (2 * i, 2 * j)
            im_ij = (2 * i + 1, 2 * j + 1)
            gigj_re = gains[re_i] * gains[re_j] + gains[im_i] * gains[im_j]
            gigj_im = gains[im_i] * gains[re_j] - gains[re_i] * gains[im_j]
            out[re_ij] = mat[re_ij] * gigj_re - mat[im_ij] * gigj_im
            out[im_ij] = mat[im_ij] * gigj_re + mat[re_ij] * gigj_im
    return out

@numba.njit
def apply_gains_numba_dense(mat, gains, ant1, ant2):
    out = np.zeros_like(mat)
    for i, ai in enumerate(ant1):
        for j, aj in enumerate(ant2):
            re_i = 2 * ai
            im_i = 2 * ai + 1
            re_j = 2 * aj
            im_j = 2 * aj + 1
            re_ij = (2 * i, 2 * j)
            im_ij = (2 * i + 1, 2 * j + 1)
            gigj_re = gains[re_i] * gains[re_j] + gains[im_i] * gains[im_j]
            gigj_im = gains[im_i] * gains[re_j] - gains[re_i] * gains[im_j]
            out[re_ij] = mat[re_ij] * gigj_re - mat[im_ij] * gigj_im
            out[im_ij] = mat[im_ij] * gigj_re + mat[re_ij] * gigj_im
    return out

def apply_gains_corrcal_dense(mat, gains, ant1, ant2, use_alt=False):
    out = mat.copy()
    if use_alt:
        apply_gains = corrcal.cfuncs.apply_gains_to_mat_dense_c
    else:
        apply_gains = corrcal.cfuncs.apply_gains_to_matrix
    # This is the same signature as is used in corrcal
    # See e.g. sparse_2level.apply_gains_to_mat in corrcal2
    # or body of optimize.get_chisq_dense in my version.
    apply_gains(
        out.ctypes.data,
        gains.ctypes.data,
        ant1.ctypes.data,
        ant2.ctypes.data,
        mat.shape[1] // 2,
        mat.shape[0],
    )
    return out

In [16]:
dense_scaled_cov = apply_gains_simple_dense(cov_dense, gains_dense, ant1, ant2)
dense_matches_complex = np.allclose(
    dense_scaled_cov[::2,::2] + 1j * dense_scaled_cov[1::2,1::2],
    apply_gains_numba(cov, gains, ant1, ant2),
)
simple_matches_numba_dense = np.allclose(
    apply_gains_simple_dense(cov_dense, gains_dense, ant1, ant2),
    apply_gains_numba_dense(cov_dense, gains_dense, ant1, ant2),
)
simple_matches_corrcal_dense = np.allclose(
    apply_gains_simple_dense(cov_dense, gains_dense, ant1, ant2),
    apply_gains_corrcal_dense(cov_dense, gains_dense, ant1, ant2),
)
corrcal_cfuncs_match = np.allclose(
    apply_gains_corrcal_dense(cov_dense, gains_dense, ant1, ant2, False),
    apply_gains_corrcal_dense(cov_dense, gains_dense, ant1, ant2, True),
)
(
    dense_matches_complex,
    simple_matches_numba_dense,
    simple_matches_corrcal_dense,
    corrcal_cfuncs_match,
)

(True, True, False, False)

In [17]:
dense_scaled_cov

array([[ 0.88155161,  0.        ,  0.36537187,  0.        ],
       [ 0.        , -0.16803676,  0.        , -0.63405702],
       [ 0.44901293,  0.        ,  0.55483831,  0.        ],
       [ 0.        ,  0.5112678 ,  0.        , -0.00529033]])

In [18]:
apply_gains_corrcal_dense(cov_dense, gains_dense, ant1, ant2, False)

array([[ 0.88155161, -0.16803676,  0.41579765, -0.00396459],
       [ 0.        ,  0.        , -0.00451427, -0.47344728],
       [ 0.51222638, -0.09763791,  0.55483831, -0.00529033],
       [ 0.11117524,  0.58324568,  0.        ,  0.        ]])

In [19]:
apply_gains_corrcal_dense(cov_dense, gains_dense, ant1, ant2, True)

array([[ 0.88155161, -0.16803676,  0.51222638, -0.09763791],
       [ 0.        ,  0.        , -0.11117524, -0.58324568],
       [ 0.56817934,  0.        ,  0.75817568,  0.        ],
       [ 0.        ,  0.64695643,  0.        ,  0.        ]])

In [20]:
# Some timing tests

In [21]:
%%timeit
_ = apply_gains_simple_dense(cov_dense, gains_dense, ant1, ant2)

18.6 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [22]:
%%timeit
_ = apply_gains_numba_dense(cov_dense, gains_dense, ant1, ant2)

690 ns ± 27.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [23]:
%%timeit
_ = apply_gains_corrcal_dense(cov_dense, gains_dense, ant1, ant2)

6.48 µs ± 38.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
